In [ ]:
"""
Purpose: To insert into the meshes that have been downloaded 
into the Segment and Mesh table


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import datajoint_utils as du
nucleus_table = du.configure_nucleus_table()
nucleus_table

INFO - 2021-01-27 22:14:37,725 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-27 22:14:37,726 - settings - Setting database.user to celiib
INFO - 2021-01-27 22:14:37,727 - settings - Setting database.password to newceliipass
INFO - 2021-01-27 22:14:37,730 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-27 22:14:37,731 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-27 22:14:37,742 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-27 22:14:38,005 - settings - Setting enable_python_native_blobs to True


ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.,"nucleus_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",supervoxel_id id of the supervoxel under the nucleus centroid. Equivalent to Allen: 'pt_supervoxel_id'.,volume volume of the nucleus in um^3
30.00,996,0,60464,93616,20968,0,35.1478
30.00,1833,0,56800,97280,19929,0,35.934
30.00,1841,0,57536,105584,19883,0,265.586
30.00,1896,0,59872,96608,19853,0,174.558
30.00,1998,864691136050815731,59936,105872,20078,72978435697419638,137.669
30.00,2005,0,61472,96128,20003,0,71.6715
30.00,2020,0,62448,97920,20177,0,214.823
30.00,2026,0,60832,100752,19857,0,103.014
30.00,2051,0,61968,96704,20133,0,38.5804
30.00,2198,864691135084884330,65088,104160,20059,73752285724957558,76.0236


In [ ]:
252832,266304,26216

In [27]:
nucleus_table & dict(segment_id=864691133080223157)

ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.,"nucleus_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",supervoxel_id id of the supervoxel under the nucleus centroid. Equivalent to Allen: 'pt_supervoxel_id'.,volume volume of the nucleus in um^3
30.00,441339,864691133080223157,252832,266304,26216,99528962253908351,43.8166


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-27 22:14:42,315 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-27 22:14:42,618 - settings - Setting enable_python_native_blobs to True


# Figuring Out What tables we dont have

In [5]:
import datajoint as dj
version = 30
m65mat = dj.create_virtual_module('m65mat', 'microns_minnie65_materialization')
nucleus_table_ids = m65mat.Nucleus.Info & {'ver': version} & "segment_id>0"

In [16]:
changed_ids = nucleus_table_ids.proj() - minnie.Mesh.proj()
changed_ids = nucleus_table_ids.proj() - minnie.Segment.proj()

In [17]:
need_to_download_mesh_ids = changed_ids.fetch("segment_id")
segmnets_to_process = need_to_download_mesh_ids[need_to_download_mesh_ids!= 0]
segmnets_to_process.shape

(25623,)

In [9]:
minnie.NucleiSegmentsRun2()

segment_id segment id for those to be decimated
864691131559524855
864691131658969700
864691131705161142
864691131771477291
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131906133483


In [25]:
minnie.NucleiSegmentsRun2 - (minnie.Decimation() & "decimation_ratio=0.25").proj()

segment_id segment id for those to be decimated
864691131658969700
864691131705161142
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131916334184
864691131942873927
864691132023987251


In [21]:
key_source = (minnie.Mesh.proj() * (minnie.DecimationConfig & 'decimation_ratio=0.25')) & minnie.NucleiSegmentsRun2.proj()
key_source

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691131559524855,0,0.25
864691131771477291,0,0.25
864691131906133483,0,0.25
864691131927301557,0,0.25
864691131953361537,0,0.25
864691131956238489,0,0.25
864691131993845256,0,0.25
864691132002968387,0,0.25
864691132050580711,0,0.25
864691132067905637,0,0.25


In [ ]:
minnie.Me

In [11]:
meshes_still_not_present = minnie.NucleiSegmentsRun2() -  minnie.Mesh.proj()
meshes_still_not_present

segment_id segment id for those to be decimated
864691131658969700
864691131705161142
864691131800703080
864691131820044899
864691131828706323
864691131834278152
864691131848667157
864691131916334184
864691131942873927
864691132023987251
